In [ ]:
#如果尚未安装，请去掉下一行的注释，并运行
#!pip install tqdm

In [16]:
## 慎用
import warnings
warnings.filterwarnings('ignore')

##数据集
import sklearn.datasets as ds
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from IPython.display import display

## 载入iris数据

In [30]:
iris  = ds.load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
display(iris_df.head())
display(iris_df.shape)
display(iris_df.describe())

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


(150, 4)

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


## 一些简单的基于iterable对象的例子

In [ ]:
from tqdm import tqdm_notebook, _tqdm_notebook
_tqdm_notebook.tqdm_notebook.pandas()

In [ ]:
a = list(range(1, 10000))
b = range(1, 10000)
_ = [(lambda x: x+1)(i) for i in tqdm_notebook(a)]
_ = [(lambda x: x+1)(i) for i in tqdm_notebook(b)]

In [ ]:
from tqdm._tqdm_notebook import tnrange

In [ ]:
_ = [(lambda x: x+1)(i) for i in tnrange(1, 10000)]

In [ ]:
for i in tnrange(1, 10, desc='i Loop'):
    for j in tnrange(1, 10000, desc='j Loop'):
        i+j

In [ ]:
for i in tqdm_notebook(range(3), desc='i-Loop'):
    for j in tqdm_notebook(range(1, 100000), desc='j-Loop'):
        i+j

In [ ]:
for i in tqdm_notebook(range(100), desc='i-Loop'):
    for j in tqdm_notebook(range(10000), desc='j-Loop', leave=False):
        i+j

## 多核的例子

In [5]:
from multiprocessing import Pool

In [19]:
def f(x):
    return spearmanr(np.linspace(1, 10, x), np.logspace(1, 10, x))
p = Pool(2)

In [20]:
_ = [i for i in p.imap(f, tnrange(3, 1000))]

In [25]:
_ = [i for i in tqdm_notebook(p.imap(f, range(3, 1000)))]

In [26]:
_ = [i for i in tqdm_notebook(p.imap(f, range(3, 1000)), total=997)]

## Pandas使用

In [33]:
_ = iris_df['sepal_length'].progress_apply(lambda x: x+1)

In [38]:
_ = iris_df.groupby(by='sepal_length').progress_apply(lambda x: x+1)

In [54]:
def parallelize_dataframe(df, func, n_jobs=3, split_num=10):
    ## 拆分数据表
    df_split = np.array_split(df, split_num)
    pool = Pool(n_jobs)
    df_list = []
    
    ## map操作
    for df_element in tqdm_notebook(pool.imap(func, df_split), total=split_num):
        df_list.append(df_element)
       
    ## reduce操作
    df = pd.concat(df_list)
    
    ## 关闭进程
    pool.close()
    pool.join()
    return df

In [55]:
def apply_add_1(df):
    return df.apply(lambda row: row['sepal_length']+1, axis=1)
_ = parallelize_dataframe(iris_df, apply_add_1)